In [1]:
pip install opencv-python opencv-python-headless opencv-contrib-python numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import os
import numpy as np

# Directories
input_dir = r"E:/Cropped_Faces"  # Root directory containing subdirectories of frames
output_dir = r"E:/Optical_Flow"  # Directory to save optical flow images

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to normalize and save optical flow images
def save_optical_flow_images(flow, output_path_x, output_path_y):
    # Split the flow into x and y components
    flow_x, flow_y = flow[..., 0], flow[..., 1]

    # Normalize the flow to 0-255 for visualization
    flow_x_norm = cv2.normalize(flow_x, None, 0, 255, cv2.NORM_MINMAX)
    flow_y_norm = cv2.normalize(flow_y, None, 0, 255, cv2.NORM_MINMAX)

    # Convert to uint8
    flow_x_norm = np.uint8(flow_x_norm)
    flow_y_norm = np.uint8(flow_y_norm)

    # Save the images
    cv2.imwrite(output_path_x, flow_x_norm)
    cv2.imwrite(output_path_y, flow_y_norm)

# Function to compute optical flow between two frames
def compute_optical_flow(frame1_path, frame2_path, output_dir):
    # Read the two consecutive frames in grayscale
    frame1 = cv2.imread(frame1_path, cv2.IMREAD_GRAYSCALE)
    frame2 = cv2.imread(frame2_path, cv2.IMREAD_GRAYSCALE)

    if frame1 is None or frame2 is None:
        print(f"[ERROR] Could not read frames: {frame1_path} or {frame2_path}")
        return

    # Compute dense optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(
        frame1, frame2, None,
        pyr_scale=0.5, levels=3, winsize=15, iterations=3,
        poly_n=5, poly_sigma=1.2, flags=0
    )

    # Generate filenames for the flow images
    base_name1 = os.path.splitext(os.path.basename(frame1_path))[0]
    base_name2 = os.path.splitext(os.path.basename(frame2_path))[0]
    output_path_x = os.path.join(output_dir, f"{base_name1}_to_{base_name2}_flow_x.jpg")
    output_path_y = os.path.join(output_dir, f"{base_name1}_to_{base_name2}_flow_y.jpg")

    # Save optical flow images
    save_optical_flow_images(flow, output_path_x, output_path_y)

# Traverse through each subdirectory
for subdir in os.listdir(input_dir):
    subdir_path = os.path.join(input_dir, subdir)
    if not os.path.isdir(subdir_path):
        continue  # Skip if not a directory

    # Output directory for the current subdirectory
    sub_output_dir = os.path.join(output_dir, subdir)
    os.makedirs(sub_output_dir, exist_ok=True)

    # Get sorted list of frames in the subdirectory
    frame_files = sorted([f for f in os.listdir(subdir_path) if f.endswith((".jpg", ".png"))])

    # Ensure there are at least two frames for optical flow computation
    if len(frame_files) < 2:
        print(f"[WARNING] Not enough frames in {subdir_path} to compute optical flow.")
        continue

    # Compute optical flow for consecutive frames
    for i in range(len(frame_files) - 1):
        frame1_path = os.path.join(subdir_path, frame_files[i])
        frame2_path = os.path.join(subdir_path, frame_files[i + 1])

        # Compute optical flow
        compute_optical_flow(frame1_path, frame2_path, sub_output_dir)

    print(f"[INFO] Completed optical flow for subdirectory: {subdir}")

print("[INFO] Optical flow computation completed for all subdirectories.")


[INFO] Completed optical flow for subdirectory: M004_light_up_angry_camera_front
[INFO] Completed optical flow for subdirectory: M004_light_up_angry_camera_right
[INFO] Completed optical flow for subdirectory: M004_light_up_angry_camera_down
[INFO] Completed optical flow for subdirectory: M004_light_up_angry_camera_left
[INFO] Completed optical flow for subdirectory: M004_light_up_angry_camera_leftfront
[INFO] Completed optical flow for subdirectory: M004_light_up_angry_camera_rightfront
[INFO] Completed optical flow for subdirectory: M004_light_up_angry_camera_up
[INFO] Completed optical flow for subdirectory: M004_light_rightup_angry_camera_front
[INFO] Completed optical flow for subdirectory: M004_light_rightup_angry_camera_right
[INFO] Completed optical flow for subdirectory: M004_light_rightup_angry_camera_down
[INFO] Completed optical flow for subdirectory: M004_light_rightup_angry_camera_left
[INFO] Completed optical flow for subdirectory: M004_light_rightup_angry_camera_leftfro